In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 198 kB 55.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=3c85db2a3691155b0d2a7236a89d74b1811ab3dbf57f11ff8faa20a090a15dfa
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Logistic Regression Model").getOrCreate()
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.sql import functions as f
import time

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()
df = spark.read.csv('/content/drive/MyDrive/Datasets/Thesis/dr16.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- objid: double (nullable = true)
 |-- modelMag_u: double (nullable = true)
 |-- modelMag_g: double (nullable = true)
 |-- modelMag_r: double (nullable = true)
 |-- modelMag_i: double (nullable = true)
 |-- modelMag_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- redshift: double (nullable = true)
 |-- zerr: double (nullable = true)
 |-- mmug: double (nullable = true)
 |-- mmgr: double (nullable = true)
 |-- mmri: double (nullable = true)
 |-- mmiz: double (nullable = true)
 |-- mfug: doub

In [ ]:
df = df.drop('objid', '_c0')
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+---------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z| redshift|             zerr|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+---------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|  2

In [ ]:
print(df.count(), len(df.columns))

3524477 27


In [ ]:
### Remove Duplicate rows in Dataframe
df = df.dropDuplicates()
print(df.count(), len(df.columns))

3524477 27


In [ ]:
import pyspark.sql.functions as func

df = df.withColumn("redshift", func.round(df["redshift"], 2).cast('integer'))

In [ ]:
import pandas as pd
pd.DataFrame(df.take(5), columns = df.columns).transpose()

,0,1,2,3,4
modelMag_u,21.322890,21.540440,24.099690,19.703640,21.891370
modelMag_g,21.199790,21.344430,21.893860,17.857980,19.901890
modelMag_r,20.978130,20.455390,20.295770,16.785280,18.379160
modelMag_i,20.786010,19.635970,19.441590,16.337970,17.835950
modelMag_z,20.602870,19.201110,19.201700,16.005240,17.496330
fiberMag_u,21.721530,22.420790,24.791400,20.956700,22.894670
fiberMag_g,21.571780,22.167020,22.645580,19.119800,20.870210
fiberMag_r,21.418020,21.318560,21.237360,18.061850,19.368600
fiberMag_i,21.290620,20.557740,20.370210,17.609200,18.812600
fiberMag_z,21.389900,20.050190,20.087840,17.276210,18.468090


In [ ]:
df.show(5)

+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------+---------+---------+------------------+---------+---------+---------+
|modelMag_u|        modelMag_g|modelMag_r|        modelMag_i|        modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|       petroR90_z|       r|                 i|                 z|redshift|                zerr|     mmug|     mmgr|     mmri|     mmiz|              mfug|     mfgr|     mfri|     mfiz|
+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------+---------+-

In [ ]:
df.columns

['modelMag_u',
 'modelMag_g',
 'modelMag_r',
 'modelMag_i',
 'modelMag_z',
 'fiberMag_u',
 'fiberMag_g',
 'fiberMag_r',
 'fiberMag_i',
 'fiberMag_z',
 'petroR50_r',
 'petroR90_r',
 'petroR50_z',
 'petroR90_z',
 'r',
 'i',
 'z',
 'redshift',
 'zerr',
 'mmug',
 'mmgr',
 'mmri',
 'mmiz',
 'mfug',
 'mfgr',
 'mfri',
 'mfiz']

In [ ]:
#df.show(5)

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("objid", monotonically_increasing_id())
df.show(5)

+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------+---------+---------+------------------+---------+---------+---------+-----+
|modelMag_u|        modelMag_g|modelMag_r|        modelMag_i|        modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|       petroR90_z|       r|                 i|                 z|redshift|                zerr|     mmug|     mmgr|     mmri|     mmiz|              mfug|     mfgr|     mfri|     mfiz|objid|
+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz'], outputCol = 'features')

In [ ]:
output = assembler.transform(df)

In [ ]:
final_df = output.select('features', 'redshift')
final_df.show(3)

+--------------------+--------+
|            features|redshift|
+--------------------+--------+
|[21.32289,21.1997...|       2|
|[21.54044,21.3444...|       0|
|[24.09969,21.8938...|       0|
+--------------------+--------+
only showing top 3 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])

In [ ]:
classifier = LogisticRegression(labelCol = 'redshift', featuresCol = 'features')

In [ ]:
start_time = time.time()
model = classifier.fit(train)
print("%s seconds" % (time.time() - start_time))

439.68651008605957 seconds


In [ ]:
pred_data = model.transform(test)

In [ ]:
pred_data.head(1)

[Row(features=DenseVector([13.6842, 11.889, 11.0987, 10.7008, 10.4458, 17.821, 15.8097, 14.9329, 14.4994, 14.2255, 21.1807, 57.1594, 17.7503, 47.1944, 11.0987, 10.7008, 10.4458, 12.7866, 11.4939, 10.8997, 10.5733, 16.8153, 15.3713, 14.7162, 14.3625]), redshift=0, rawPrediction=DenseVector([44.1393, 16.7517, -12.1266, -38.9708, -31.3665, 9.3028, 15.1299, -2.8599]), probability=DenseVector([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]), prediction=0.0)]

In [ ]:
evaluator = MulticlassClassificationEvaluator(metricName = 'accuracy',labelCol = 'redshift')

In [ ]:
AUC = evaluator.evaluate(pred_data)

In [ ]:
print(AUC)

0.8917375874795035


In [ ]:
print('test data (weightedPrecision): ', evaluator.setMetricName('weightedPrecision').evaluate(pred_data))
print('test data (weightedRecall): ', evaluator.setMetricName('weightedRecall').evaluate(pred_data))

test data (weightedPrecision):  0.8895100973804296
test data (weightedRecall):  0.8917375874795034


In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol = 'redshift')
predictionAndTarget = model.transform(final_df).select("redshift", "prediction")
acc = evaluator.evaluate(predictionAndTarget, {evaluator.metricName: "accuracy"})
print(acc)

0.8918642397155663


# Cascading

In [ ]:
pred_data2 = model.transform(final_df)

In [ ]:
pred_data2.count()

3524477

In [ ]:
pred_data2.show(5)

+--------------------+--------+--------------------+--------------------+----------+
|            features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[21.32289,21.1997...|       2|[2.82891209690775...|[0.12433162639974...|       1.0|
|[21.54044,21.3444...|       0|[6.60756029169412...|[0.92606750232092...|       0.0|
|[24.09969,21.8938...|       0|[6.74758155424796...|[0.99493005913909...|       0.0|
|[19.70364,17.8579...|       0|[8.72007909819839...|[0.99750849911570...|       0.0|
|[21.89137,19.9018...|       0|[7.43703470704134...|[0.99673433458083...|       0.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
pred_data2 = pred_data2.withColumn("objid", monotonically_increasing_id())
pred_data2.show(5)

+--------------------+--------+--------------------+--------------------+----------+-----+
|            features|redshift|       rawPrediction|         probability|prediction|objid|
+--------------------+--------+--------------------+--------------------+----------+-----+
|[21.32289,21.1997...|       2|[2.82891209690775...|[0.12433162639974...|       1.0|    0|
|[21.54044,21.3444...|       0|[6.60756029169412...|[0.92606750232092...|       0.0|    1|
|[24.09969,21.8938...|       0|[6.74758155424796...|[0.99493005913909...|       0.0|    2|
|[19.70364,17.8579...|       0|[8.72007909819839...|[0.99750849911570...|       0.0|    3|
|[21.89137,19.9018...|       0|[7.43703470704134...|[0.99673433458083...|       0.0|    4|
+--------------------+--------+--------------------+--------------------+----------+-----+
only showing top 5 rows



In [ ]:
print(df.count(), len(df.columns))
print(pred_data2.count(), len(pred_data2.columns))

new_df = df.join(pred_data2, df.objid == pred_data2.objid).select(df["*"],pred_data2["prediction"])
#new_df = df.join(dt_predictions, df.objid == dt_predictions.objid)
new_df.show(5)

3524477 28
3524477 6
+----------+----------+----------+----------+------------------+----------+----------+----------+----------+----------+------------------+----------+------------------+------------------+--------+--------+------------------+--------+-----------+---------+---------+------------------+---------+------------------+---------+---------+--------+-----+----------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|        modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|        petroR90_z|       r|       i|                 z|redshift|       zerr|     mmug|     mmgr|              mmri|     mmiz|              mfug|     mfgr|     mfri|    mfiz|objid|prediction|
+----------+----------+----------+----------+------------------+----------+----------+----------+----------+----------+------------------+----------+------------------+------------------+--------+--------+------------------+--------+-----------+---------+----

In [ ]:
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz', 'prediction'], outputCol = 'features2')

In [ ]:
output = assembler.transform(new_df)

In [ ]:
final_df = output.select('features2', 'redshift')
final_df.show(3)

+--------------------+--------+
|           features2|redshift|
+--------------------+--------+
|[21.32289,21.1997...|       2|
|[20.66357,20.0293...|       0|
|[21.01588,20.8195...|       1|
+--------------------+--------+
only showing top 3 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])

In [ ]:
classifier = LogisticRegression(labelCol = 'redshift',featuresCol = 'features2')

In [ ]:
model2 = classifier.fit(train)

In [ ]:
pred_data2 = model2.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(metricName = 'accuracy',labelCol = 'redshift')
AUC = evaluator.evaluate(pred_data2)
print(AUC)

0.8918099214377776


In [ ]:
print('test data (weightedPrecision): ', evaluator.setMetricName('weightedPrecision').evaluate(pred_data2))
print('test data (weightedRecall): ', evaluator.setMetricName('weightedRecall').evaluate(pred_data2))

test data (weightedPrecision):  0.8903175797479013
test data (weightedRecall):  0.8918099214377776
